In [ ]:
import kagglehub
irkaal_foodcom_recipes_and_reviews_path = kagglehub.dataset_download('irkaal/foodcom-recipes-and-reviews')
print('Data source import complete.')

Data source import complete.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [46]:
recipes=pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/recipes.csv')
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [47]:
recipes.isnull().sum()

,0
RecipeId,0
Name,0
AuthorId,0
AuthorName,0
CookTime,82545
PrepTime,0
TotalTime,0
DatePublished,0
Description,5
Images,1


In [48]:
recipes.drop(["RecipeServings", "RecipeYield", "AggregatedRating", "ReviewCount"], inplace=True, axis=1)
recipes.fillna("-", inplace=True)

In [49]:
recipes_util = recipes.copy()
columns = ['RecipeId', 'Name', 'RecipeCategory','RecipeIngredientParts','Calories','FatContent','SaturatedFatContent','CholesterolContent','SodiumContent','CarbohydrateContent','FiberContent','SugarContent','ProteinContent']
recipes_util = recipes_util[columns]

In [50]:
#Valores nutricionales "sanos" y restricciones alimentarias, pueden ser adaptables según el usuario
#https://www-verywellfit-com.translate.goog/what-does-based-on-a-2000-calorie-diet-mean-4099137?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=sge#:~:text=Based%20on%20the%20DV%20information,or%201200%20calories%20from%20carbohydrates

Daily_values = {"Calories": 2000,
                "FatContent": 78,
                "SaturatedFatContent": 20,
                "CholesterolContent": 300,
                "SodiumContent": 2300,
                "CarbohydrateContent": 275,
                "FiberContent": 28,
                "SugarContent": 40,
                "ProteinContent": 50}

Meal_values = Daily_values.copy()
for value in Meal_values:
    Meal_values[value] = Meal_values[value]/3 #Son 3 comidas al día

Restrictions = ["cream", "milk", "yogurt", "cheese"] #Intolerancia a la lactosa

In [51]:
for key, value in Meal_values.items():
    recipes_util = recipes_util[recipes_util[key] <= value]

for restriction in Restrictions:
    recipes_util = recipes_util[~recipes_util['RecipeIngredientParts'].str.contains(restriction, case=False, regex=False)]

**KNN**

In [52]:
scaler = StandardScaler()
recipes_scaled = scaler.fit_transform(recipes_util.iloc[:,4:12].to_numpy())

In [53]:
NN = NearestNeighbors(metric='cosine')
NN.fit(recipes_scaled)

transformer = FunctionTransformer(NN.kneighbors,kw_args={'return_distance':False})
pipeline = Pipeline([('std_scaler',scaler),('NN',transformer)])

In [55]:
params={'n_neighbors':10,'return_distance':False}
pipeline.set_params(NN__kw_args=params)
recs = pipeline.transform(recipes_util.iloc[0:1,4:12].to_numpy())

In [56]:
recipes_util.iloc[recs[0]]

,RecipeId,Name,RecipeCategory,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
8,46,A Jad - Cucumber Pickle,Vegetable,"c(""rice vinegar"", ""haeo"")",4.3,0.0,0.0,0.0,0.7,1.1,0.2,0.2,0.1
355777,368835,Low Fat White Bean Dip,Beans,"c(""garlic cloves"", ""lemon"", ""cumin"", ""cayenne ...",5.3,0.1,0.0,0.0,1.5,1.1,0.2,0.2,0.2
220951,230310,Fresh Strawberry Limeade,Beverages,"c(""water"", ""limes"", ""strawberries"", ""Splenda s...",2.6,0.0,0.0,0.0,0.9,0.8,0.2,0.2,0.1
68240,72847,Tarragon Chicken Salad,Chicken,"c(""chicken breasts"", ""onion"", ""celery"", ""Hellm...",6.1,0.1,0.0,0.0,3.4,1.2,0.2,0.3,0.3
507425,525902,Lil' A** Kicker,< 15 Mins,"c(""triple sec"", ""vodka"", ""lime wedge"")",2.6,0.0,0.0,0.0,0.2,0.9,0.2,0.1,0.1
183917,192225,Apple Sparkler,Beverages,"c(""lime wedges"", ""sparkling apple cider"")",2.6,0.0,0.0,0.0,0.2,0.9,0.2,0.1,0.1
489982,508044,Mini Margarita Shots,Beverages,"""lime wedge""",2.6,0.0,0.0,0.0,0.2,0.9,0.2,0.1,0.1
515892,534519,Lime Mayo for Fish Tacos,Sauces,"c(""mayonnaise"", ""lime"")",2.5,0.0,0.0,0.0,0.2,0.9,0.2,0.1,0.1
175457,183515,Garlic Seasoning,< 15 Mins,"""dried garlic""",5.9,0.0,0.0,0.0,0.5,1.3,0.2,0.3,0.3
480787,498502,Garlic Mayonnaise,Very Low Carbs,"c(""mayonnaise"", ""garlic powder"", ""lemon juice""...",3.6,0.0,0.0,0.0,1.0,0.8,0.2,0.1,0.2
